# Electoral College Analysis

The number of electoral votes is based largely on the number of seats in Congress.  Most people understand how the size of the US Senate is determined-- 2 senators per state X 50 states == 100 senators.

The size of the House of Representatives needs a bit more explaining.

## The House of Representatives

The House of Representatives is meant to be proportional in size to the population of the state.

At the start of the Uninted States, there were only 65 seats in the House of Representatives.

George Washington considered the ideal ratio of representatives to voters to be close to 1:30,000.

As of 2016, the *average* ratio is closer to 1:700,000!

### The Reappointment Act of 1929
A combined census and reapportionment bill that sets the method for apportioning seats in the U.S. House of Representatives according to each census such that there are always 435 seats.

A truly proportional division of seats would produce fractional seats, which doesn't work well for voting scenarios.
But since the stakes are high, a formal process needs to be in place.  The process, called "the method of equal proportions" is:

* Every state gets 1 seat to start.  This leaves 538 - 50 == 385 seats.
* The *priority number* for each state is computed.
* The state with the highest priority number is awarded a seat from those remaining.
* The priority number for each state is recomputed, and the process repeats until all the seats have been awarded.

The priority number for a state is computed as follows:
$$A_n = \frac{P}{\sqrt{(n(n+1)}}$$

Where $A_n$ is the priority number for a state that has $n$ seats, and $P$ is the population of the state.  The starting priority for each state is therefore:
$$A_1 = \frac{P}{\sqrt{2}}$$


## Electoral College Votes per State

* Each state gets 1 EC vote for each member of congress.
* 100 senators + 435 representatives + 3 for the District of Columbia (see the 23rd Amendment) = 538 electoral votes

    


  

In [98]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib notebook

In [99]:
pd.options.display.float_format = '{:,.2f}'.format

# The 2010 Census

Data taken from:  https://www.census.gov/popest/data/datasets.html

Section "2010 Census Modified Race Data Summary File"

In [100]:
df = pd.read_csv("STCO-MR2010_AL_MO.csv", encoding="latin-1")
df = df[["STNAME", "RESPOP"]]
df = df.groupby("STNAME").sum()
df.head()

df2 = pd.read_csv("STCO-MR2010_MT_WY.csv", encoding="latin-1")
df2 = df2[["STNAME", "RESPOP"]]
df2 = df2.groupby("STNAME").sum()
df2.head()

pops = df.append(df2)
del df
del df2
pops = pops.drop(["District of Columbia"])
print(pops.head())
print(pops.tail())

              RESPOP
STNAME              
Alabama      4779736
Alaska        710231
Arizona      6392017
Arkansas     2915918
California  37253956
                RESPOP
STNAME                
Virginia       8001024
Washington     6724540
West Virginia  1852994
Wisconsin      5686986
Wyoming         563626


In [101]:
total_pop = pops.sum()["RESPOP"]
print("Total US Population: {:,}".format(total_pop))

Total US Population: 308,143,815


If House seats were divided exactly proportionally to state populations ...

In [102]:
pops["True Proportion"] = (pops["RESPOP"] / total_pop) * 435.0
pops.head()

,RESPOP,True Proportion
STNAME,,
Alabama,4779736,6.75
Alaska,710231,1.00
Arizona,6392017,9.02
Arkansas,2915918,4.12
California,37253956,52.59


## Implementation of the method of equal proportions

In [103]:
import math

states = list(pops.index)
state_seats = {}
for state in states:
    state_seats[state] = 1
seats_remaining = 435 - 50
while seats_remaining > 0:
    max_priority = -1
    winning_state = None
    for state in states:
        pop = pops.get_value(state, "RESPOP")
        n = state_seats[state]
        priority = pop / math.sqrt(n * (n + 1))
        if priority > max_priority:
            max_priority = priority
            winning_state = state
    n = state_seats[winning_state]
    n = n + 1
    seats_remaining = seats_remaining - 1
    state_seats[winning_state] = n

### Convert the results from above into a DataFrame

In [104]:
d = dict([(state, [state_seats[state]]) for state in states])
awarded = pd.DataFrame.from_dict(d, orient='index')
awarded.rename(columns={0: "Seats"}, inplace=True)
awarded.head()

,Seats
Minnesota,8
Rhode Island,2
Colorado,7
New York,27
Georgia,14


### Join the results to the main DataFrame

In [105]:
df = pd.merge(pops, awarded, left_index=True, right_index=True)
df.head()

,RESPOP,True Proportion,Seats
Minnesota,5303925,7.49,8
Rhode Island,1052567,1.49,2
Colorado,5029196,7.10,7
New York,19378102,27.36,27
Georgia,9687653,13.68,14


### A quick check to make sure the seats and fractions both add up to 435 ...

In [106]:
df.sum()

RESPOP            308,143,815.00
True Proportion           435.00
Seats                     435.00
dtype: float64

### Add the senate seats to get the EC votes per state

In [109]:
df["EC Votes"] = df["Seats"] + 2
df["Fraction EC Votes"] = df["EC Votes"] / 535.0
df["Fraction Total Pop"] = df["RESPOP"] / total_pop
df

,RESPOP,True Proportion,Seats,EC Votes,Fraction EC Votes,Fraction Total Pop
Minnesota,5303925,7.49,8,10,0.02,0.02
Rhode Island,1052567,1.49,2,4,0.01,0.00
Colorado,5029196,7.10,7,9,0.02,0.02
New York,19378102,27.36,27,29,0.05,0.06
Georgia,9687653,13.68,14,16,0.03,0.03
Iowa,3046355,4.30,4,6,0.01,0.01
Wyoming,563626,0.80,1,3,0.01,0.00
Maine,1328361,1.88,2,4,0.01,0.00
Washington,6724540,9.49,10,12,0.02,0.02
Nebraska,1826341,2.58,3,5,0.01,0.01


### Another tally so we can make sure the fractions add up.


In [110]:
df.sum()

RESPOP               308,143,815.00
True Proportion              435.00
Seats                        435.00
EC Votes                     535.00
Fraction EC Votes              1.00
Fraction Total Pop             1.00
dtype: float64